<a href="https://colab.research.google.com/github/WhiteNoyse/gplvm/blob/master/Non_linearDimensionalityReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Non-linear Dimensionality Reduction using the Gaussian Process Latent Variable Model

### Emmanuelle Dankwa, Natalia Garcia Martin, Deborah Sulem 

## Introduction

 The Gaussian Process Latent Variable Model (GPLVM) (Lawrence 2005) is a non-linear technique of dimensionality reduction, that can be used in particular when high-dimensional data is assumed to have a low dimensional fully-expressive embedding. It is a unsupervised, non-linear method, taking advantage of the Gaussian Process regression framework, extending the Dual Probabilistic Principal Component Analysis (DPPCA) to non-linear kernels. 
 
 In this study, we analyse the GPLVM and compare its perfomance with other dimensionality reduction techniques. Comparisons are made based on computation times, classification error and 'trustworthiness'. We shall explore these measures in more detail in the report.
 
 We shall proceed as follows: in section 1, we explain the concept of Gaussian processes and present the general concept of the Principal Component Analysis (PCA) procedure in section 2. The theoretical basis of GPLVM is discussed in section 3 and a variational approach (Titsias and Lawrence, 2010) is considered. We then outline five other dimensionality reduction measures in  section 4. Subsequently, we study the performances of all 6 different dimensionality reduction techniques on an artificial dataset. Also, to observe how the methods perform on real data, we apply each of them to an interesting twelve-dimensional data set describing oil flow in a pipeline (Bishop et al., 1998).  Details and results of these comparisons are given in section 5. We conclude the study with a discussion in section 6 outlining  major observations and suggesting further research areas. 


## 1.   Gaussian Process

A Gaussian process (GP) is defined as a collection of random variables, any finite number of which have a joint (multivariate) Gaussian distribution. In Gaussian process regression, we assume that the output $y$ of a function $f$ at input $\boldsymbol x$ can be written as $y = f(\boldsymbol x) + \epsilon$ with $\epsilon \sim N (0, \sigma_\epsilon^2)$. The GP approach is a non-parametric approach, in that it finds a distribution over the possible functions $f(\boldsymbol x)$ that are consistent with the observed data. We assume that the function $f(\boldsymbol x)$ is distributed as a Gaussian process:
$$f(\boldsymbol x) ∼ GP (m(\boldsymbol x), k(\boldsymbol x, \boldsymbol x')).$$

A Gaussian process GP is a distribution over functions and is defined by a mean and a covariance function. The mean function $m(x)$ reflects the expected function value at input $x$: $m(x) = E[f(x)]$. The prior mean function is often set to $m(x) = 0$ in order to avoid expensive posterior computations and only do inference via the covariance function. We can do so by subtracting the (prior) mean from all observations. The covariance function $k(\boldsymbol x, \boldsymbol x^T)$, often referred to as the kernel of the GP, models the dependence between the function values at different input points $\boldsymbol x$ and $\boldsymbol x'$:
$$k(\boldsymbol x, \boldsymbol x')=E\left[\left(f(\boldsymbol x)-m(\boldsymbol x)\right)\left( f(\boldsymbol x')-m(\boldsymbol x')\right)\right].$$

The choice of an appropriate kernel is based on assumptions such as smoothness and likely patterns to be expected in the data. A sensible assumption is usually that the correlation
between two points decays with the distance between the points: closer points are expected to behave more similarly than points which are further away. The Radial Basis Function (RBF) kernel fulfills this assumption and can be used to model smooth and stationary functions. It is defined as 

$$k(\boldsymbol x, \boldsymbol x')= σ^2_f \exp \left( \frac{-|| \boldsymbol x- \boldsymbol x'||^2}{2 \lambda^2} \right).$$

The two hyper-parameters $λ$ (the lengthscale) and $σ^2_f$ (the signal variance) can be varied to increase or reduce the a priori correlation between points and consequentially the variability of the resulting function. Once a mean function and kernel are chosen, we can use the Gaussian process to draw a priori function values, as well as posterior function values conditional upon previous observations <cite>[1]</cite>.



## 2. Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a linear technique for dimensionality reduction. The key is to construct a lower-dimensional representation of the data that describes as much of the variance in the data as possible. We will denote the $D$-dimensional observed data by $X\in \mathbb{R}^{~N \times D}$ and the lower-dimensional representation in the Q-dimensional space by $Y \in \mathbb{R}^{~N \times Q}$. This is done by finding a low dimensional linear basis that maximises the amount of variance in the data: a linear mapping $\boldsymbol{M}$ that maximises the cost function $\text{trace}(\boldsymbol{M}^T\text{cov}(\boldsymbol{X})\boldsymbol{M})$, where $\text{cov}(\boldsymbol{X})$ is the sample covarance matrix of the data $\boldsymbol{X}$. This is given by the first $Q$ principal eigenvectors (principal components) of the sample covariance matrix of the zero-mean data. We want to maximise $\boldsymbol{M}^T\text{cov}(\boldsymbol{X})\boldsymbol{M}$ with respect to $\boldsymbol{M}$  with the constraint that the L2-norm of each column of $\boldsymbol{M}$ is 1, that is $||\boldsymbol{m_j}||^2=1.$ We can introduce this constraint by introducing the Lagrange multiplier $\lambda$ such that we perform the unconstrained maximisation of 

$$\boldsymbol{m_j}^T\text{cov}(\boldsymbol{X})\boldsymbol{m_j} + \lambda (1-\boldsymbol{m_j^T}\boldsymbol{m_j}).$$

The stationary points are given by $\text{cov}(\boldsymbol{X})\boldsymbol{m_j}=\lambda \boldsymbol{m_j}.$ Therefore, PCA solves the eigenprolem 

$$\text{cov}(\boldsymbol{X})\boldsymbol{M}=\lambda \boldsymbol{M} $$

for the $Q$ principal eigenvalues $\lambda$ and the lower-dimensional representation of the data is then given by $\boldsymbol{Y}=\boldsymbol{XM}$. Applications of PCA include face recognition and seismic analysis. However, a drawback of PCA is that the size of the covariance matrix is proportional to the dimensionality of the data, making the computation of the eigenvectors infeasible for very high-dimensional data. This can be overcome by using a latent-variable formulation of PCA called probabilistic PCA (Tipping and Bishop, 1999) that will be discussed in the next section. It uses a Gaussian prior over the latent space and a linear-Gaussin noise model, leading to an iterative and computationally efficient expectation-maximisation (EM) algorithm. We can further extend it to non-linear mappings (GP-LVM) by using Gaussian processes.

## 3.   Gaussian Process Latent Variable Model (GP-LVM)

### Linear probabilistic approaches: PPCA and DPPCA

In 2004, Lawrence introduces a novel probabilistic interpretation of PCA referred to as Dual Probabilistic PCA (DPPCA) which turns out to be a special case of a more general class of models referred to as GP-LVMs. Latent variable models usually relate a set of latent variables $X \in \mathbb{R}^{~N \times Q}$  to a set of observed variables $Y \in \mathbb{R}^{~N \times D}$ through a set of parameters by marginalising the latent variables and optimising the parameters. For the purpose of dimensionality reduction, $Q\ll D$. Lawrence (2004) introduces an alternative approach where the parameters are marginalised and the latent variables are then optimised to learn the lower-dimensional representation. 

Probabilistic PCA (PPCA) (Tipping and Bishop, 1999) is formulated as a latent variable model where the maximum likelihood solution of the parameters is found by solving an eigenvalue problem on the data's covariance matrix. We denote the $D$-dimensional data $\{\boldsymbol{y}_n\}_{n=1}^N$  and the associated latent variables $\{\boldsymbol{x}_n\}_{n=1}^N$. The relationship between the latent variables and the data points is given by
$$\boldsymbol{y}_n=\boldsymbol{Wx}_n+\boldsymbol{\eta}_n,$$

where $\boldsymbol{W}\in \mathbb{R}^{~D \times Q}$ specifies the linear relationship between the latent and the data space, and $\boldsymbol{\eta}_n \in \mathbb{R}^{D}$ are sampled independently from a spherical Gaussian distribution with zero mean and covariance $\beta^{-1}\boldsymbol I$: 
$$p(\boldsymbol{\eta}_n) = N (\boldsymbol{\eta}_n|\boldsymbol 0, \beta^{-1}\boldsymbol I).$$

While PPCA puts a (Gaussian) prior distribution on the latent variables, marginalize the likelihood $p(Y|X,W,\beta)$ over the latter and optimize the model parameters $\{W,\beta\}$, Dual Probabilistic PCA (DPPCA) puts a spherical Gaussian prior on $W$:
$$ p(W) = \Pi_{d=1}^D \mathcal{N}(w_d; 0, I_Q)$$
The marginal likelihood of the data is then:
$$ p(Y|X,\beta) = \Pi_{d=1}^D p(y_d|X,\beta) \quad \text{with} \quad p(y_d|X,\beta) = \mathcal{N}(y_d; 0, K) \quad \text{and} \quad K = X X^T + \frac 1 \beta I_Q$$
where $y_d$ is the d-th column of Y. Equivalently, the marginal log-likelihood is thus:
$$ L = - \frac {DN} 2 \log 2\pi - \frac N 2 \log det(K) - \frac 1 2 tr(K^{-1}Y Y^T)$$
Recovering the latent variables $X$ thus only requires to maximize the log-likelihood, solving an eigenvalue problem equivalent to PCA. 

The Gaussian Process Latent Variable Model generalises this probabilistic approach using non-linear kernels $K$ in the latent space, such as the RBF kernel.

###  Non-Linear probabilistic approaches: GP-LVM

As previously seen, the Gaussian Process Latent Variable Model (GP-LVM) is a Gaussian process mapping from a low-dimensional latent space to a high-dimensional observation space with non-linear covariance functions.The model is optimized by computing the derivative of the likelihood with respect to the latent space through a chain rule:
$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial K} \times \frac{\partial K}{\partial X} \quad \text{with} \quad \frac{\partial L}{\partial K} = K^{-1} Y Y^T K^{-1}  - D K^{-1}  $$
Provided that the derivative of the kernel matrix with respect to the latent variables can be computed, the latent variables as well as the kernel parameters can be optimized via a gradient based optimisation method. However, each step of the latter requires to compute the inverse of the kernel matrix $K^{-1} $, which has a $O(N^3)$ computational cost.

### Sparse GP-LVM

In [4], Lawrence introduces a sparse approximation of his method in order to reduce the complexity of each iteration of the optimization to $O(k^2 N)$ where $k$ is the size of the subset of points kept in the sparse representation. This method incorporates some inducing variables (or pseudo-inputs) $\boldsymbol{U} \in \mathbf{R}^{k \times D}$ and puts a Gaussian Process prior on each of the columns:
$$p(u_d) = \mathcal{N}(u_d; 0_k, K_{u_d u_d}) $$
where $ K_{u_d u_d}$ is a covariance function constructed using some latent variables $\boldsymbol{X_u}$ which can be a subset of the variables in $\boldsymbol{X}$ .

Let  $\boldsymbol{F} \in \mathbf{R}^{N \times D}$ be the function values of the observations. Each column $u_d$ is then jointly distributed with $f_d$ and:
$$ p(f_d|u_d) = \mathcal{N}(f_d; K_{f_d u_d} K_{u_d u_d}^{-1} u_d, K_{f_d f_d} - K_{f_d u_d} K_{u_d u_d}^{-1} K_{u_d f_d})$$
where $ K_{f_d f_d}$ is the covariance matrix of the inputs  $\boldsymbol{X}$ and $ K_{f_d u_d}$ and $ K_{u_d f_d}$ are the (non-symmetric) covariances matrices between the inducing variables $\boldsymbol{X_u}$ and the latent variables $\boldsymbol{X}$.

The sparse approximations rely on approximated distributions $q(f_d|u_d)$ of $ p(f_d|u_d)$.  Lawrence proposes three methods, the Deterministic Training Conditional, the Fully Independent Training Conditional and the Partially Independent Training Conditional in order to compute an approximated marginal distribution of the function values in the form:
$$q(f_d) =  \mathcal{N}(f_d; 0_N, \boldsymbol{K}')$$
where $\boldsymbol{K}'$ is a product of matrices that includes only the inverse of $K_{u_d u_d}$. The optimization of the likelihood is thus faster, due to the reduced dimension of the latter matrix.

### Bayesian GP-LVM (Titsias and Lawrence 2010)

Two main issues with the GP-LVM is that it requires to (manually) select the dimensionality Q of the latent space, and that the marginal log-likelihood over the latent variable need to be tractable. To overcome this difficulty, a non-standard variational approach was proposed by Titsias and Lawrence in [5]. They propose a new variational algorithm for the GP-LVM and provide a variational lower bound on the marginal likelihood of the data. This method speeds up the GP-LVM algorithm using the sparse approximation framework and avoids overfitting when the dimension of the latent space is not known a priori. In fact, using a specific kernel, the automatic relevance determination (ARD) squared exponential kernel,  allows to automatically infer the lower space's dimension $Q$:
$$k(x,x') = \sigma_f^2 \exp( - \frac 1 2 \sum_{q = 1}^{Q} \frac {(x_q - x_q')^2}{\lambda_q^2})$$
The difference with the RBF kernel is that the lengthscales $\{\lambda_q\}_{q=1}^Q$ depend on the dimension in the latent space. In particular, if $\lambda_q \rightarrow \infty$, then the dimension $q$ is irrevelant in the observation space. Optimizing the vector of lengthscales $\lambda = (\lambda_1, \dots, \lambda_Q)$ thus automatically peforms a dimensionality selection in the data space

Besides, this method can be used for computing the marginal likelihood of the observations and for inference of missing data.

In the context of Variational Inference, the objective is to compute the marginal likelihood of the data $p(Y) = \int p(Y|X)p(X)dX$, which is intractable in the GP-LVM. Introducing a variational distribution with a factorized Gaussian form $q(X) = \Pi_{n=1}^N \mathcal{N}(x_n; \mu_n, S_n)$ approximating the true posterior $p(Y|X)$, a Jensen's lower bound is given by:

$$ \log p(Y) \geq \int q(X) \log p(Y|X)dX - KL(q||p) = \sum_{d=1}^D \tilde{F}_d(q)  - KL(q||p)$$
using the assumption that the GPs are independent across the features, with $\tilde{F}_d(q) = \int  q(X) \log p(y_d|X)dX$. The KL divergence between the two Gaussian distributions $p(X)$ and $q(X)$ is analytical but the first term can only be lower bounded in a closed-form using a variational approximation.

Similarly to the sparse GP regression framework, this method incorporates the inducing variables and the function values and compute a variational distribution to approximate the true posterior $p(f_d,u_d|X,y_d)$:
$$q(f_d, u_d) = p(f_d|u_d,X)\phi(u_d)$$
The lower variational bound is then maximized with respect to the distribution $\phi(u_d)$, amd then jointly maximized over the model hyperparameters - the kernel parameters $\boldsymbol{\theta} = (\sigma_f, \lambda_1, \dots, \lambda_Q)$ and $\beta$ - and the variational parameters $\{\mu_n, S_n\}_{n=1}^N$ to recover the latent variables. 

 </p>




## 4. Other Dimensionality Reduction Methods

Dimensionality reduction techniques can be grouped into convex and non-convex techniques. The first optimise an objective function that does not contain any local optimum, that is, the solution space is convex, while the latter optimise objective functions that do contain local optima (van der Maaten et al., 2009). Examples of convex techniques include PCA and Kernel PCA. Both are examples of full spectral techniques, as they perform an eigendecomposition of a full matrix that captures the covariances between dimensions or the pairwise similarities between datapoints, in contrast to convex sparse spectral techniques such as LLE (local linear embedding) that make use of sparse matrices. We will look at autoencoders as an example of a non-convex approach. 

In this section we shall discuss five techniqes in all: kernel PCA, autoencoders, generative topographical mapping, multidimensional Scaling methods and t-distribution stochastic neighbour embedding; before going on to analyse their performance empirically.

### 4.1. Kernel PCA

Kernel PCA (KPCA) is a reformulation of PCA where a kernel function is used which allows us to construct non-linear mappings. Instead of computing the eigenvectors of the covariance matrix, we now compute the eigenvectors of the kernel matrix. The entries of the kernel matrix are given by 

$$k_{ij}=\kappa(\boldsymbol x_i, \boldsymbol x_j),$$

where $\kappa$ is a kernel function giving rise to a positive-semidefinite kernel $\boldsymbol K.$ The kernel matrix $\boldsymbol{K}$ is then double-centred using

$$k_{ij}=-\frac{1}{2}\left( k_{ij}-\frac{1}{n}\sum_l k_{il}-\frac{1}{n}\sum_l k_{jl}+\frac{1}{n^2}\sum_{lm} k_{lm}\right).$$ 

This process is equivalent to subtracting the mean of the features in PCA. Instead, we subtract the mean of the data in the feature space defined by the kernel function $\kappa$ so that the data has zero mean in this space. The principal $Q$ eigenvectors $\boldsymbol v_i$ of the centred kernel matrix are then computed. From these, we can obtain the eigenvectors of the covariance matrix $\boldsymbol{a_i}$ in the space spanned by $\kappa$:

$$\boldsymbol{a_i}=\frac{1}{\sqrt \lambda_i}\boldsymbol{v_i}.$$

The low-dimensional data representation $\boldsymbol{Y}$ is then obtained by projecting the data $X$ on the eigenvectors of the covariance matrix $\boldsymbol{a_i}$:

$$\boldsymbol{y_i}=\left \{ \sum_{j=1}^n a_1^{(j)}\kappa(\boldsymbol x_j, \boldsymbol x_i), \dots, \sum_{j=1}^n a_Q^{(j)}\kappa(\boldsymbol x_j, \boldsymbol x_i)\right\},$$

where $a_1^{(j)}$ indicates the $j$th value of the vector $\boldsymbol{a}_1$ and $\kappa$ is the kernel function used to construct $\boldsymbol{K}$. Some popular choices of kernel include the linear kernel, which results in standard PCA, the polynomial kernel, or the Gaussian kernel, also known as the radial basis function (RBF) kernel. While both Kernel PCA and GP-LVM both use kernel functions to construct non-linear variants of PPCA, the kernel function of the GP-LVM is defined over the low-dimensional latent space, whereas the kernel function of Kernel PCA is defined over the high-dimensional data space.

### 4.2. Autoencoders (AE)

The autoencoder (AE) neural network is an unsupervised learning algorithm that applies backpropagation, setting the target values to be equal to the inputs. A hidden layer $\boldsymbol{h}$ describes the code used to represent the input. An autoencoder generally consists of two parts: an encoder $\boldsymbol{h}=f(\boldsymbol{x})$ which transforms the input to a hidden code and a decoder $g$ which reconstructs the input from hidden code to give $\boldsymbol{r}=g(\boldsymbol{h})$. Modern autoencoders have been generalised these deterministic functions to stochastic mappings $p_{encoder}(\boldsymbol{h}|\boldsymbol{x})$ and $p_{decoder}(\boldsymbol{x}|\boldsymbol{h})$ (Goodfellow et al., 2016). Two practical applications of autoencoders are data denoising and dimensionality reduction for data visualization. 

Simply learning $g(f(\boldsymbol{x}))=\boldsymbol{x}$  is not particularliy useful. Instead, we design the autoencoder so that they copy only certain aspects of the input, hence learning key properties of the data. Autoencoders whose code dimension is less than the input dimension are called undercomplete. This forces the autoencoder to capture the most significant features of the training data. The simplest case is the Vanilla autoencoder, where we only have one hidden layer. When more than one hidden layer is present, we denote these as multilayer autoencoders. Multilayer autoencoders are feed-forward neural networks with an odd number of hidden laters and shared weights between the top and bottom layers. The middle hidden layer has $Q$ nodes, and the input and output layer have $D$ nodes. The aim is to obtain a middle hidden layer that gives a $Q$-dimensional representation of the data that preserves as much as the structure from the data $\boldsymbol{x}$ as possible. 

The learning process is described as minimising a loss function
$$L(\boldsymbol{x}, g(f(\boldsymbol{x}))),$$

where $L$ is the loss function penalising the output $g(f(\boldsymbol{x}))$ for being dissimilar from the input data $\boldsymbol{x}$. The mean squared error is usually used. Techniques for feedforward networks like minibatch gradient descent are generally used. Additionally, autoencoders may be trainbed using recirculation (Hinton and McClelland, 1988), which compares the activations of the network on the original input to those on the reconstructed output.

In order to train the autoencoder to learn a non-linear mapping between the high-dimensional and the low-dimensional data representation, sigmoid activation functions are usually employed except for in the middle layer where a linear activation function is often used (van der Maaten et al., 2009). The figure below from the paper by van der Maaten et al. gives us an overview of what the structure of the autoencoder looks like. 

In [11]:
from IPython.display import HTML, display
display(HTML("<img src='autoencoder.jpg'>"))

### 4.3. Generative Topographic Mapping (GTM)

GTM (Bishop et al., 1998) is a non-linear latent-variable model which maps from a low-dimensional latent space into a high dimensional data space via a topology-preserving map. Specifically,  the GTM employs a  non-linear function which maps from the latent space into a non-Euclidean manifold, $S$, embedded in the observed data space and of the same dimensionality as the latent space.



Consider a non-linear function $\phi(\boldsymbol{x};W)$ from the latent space, $X$ to the data space $Y$, where $W$ is a parameter matrix of weights. Let $\phi(\boldsymbol{x};W) = \boldsymbol{q}$. Then, the distribution of $\boldsymbol{y}$ given $\boldsymbol{x}$ and $W$ is chosen to be Gaussian : 


$$p(\boldsymbol{y}|\boldsymbol{x}, \boldsymbol{W}, \beta) = \left(\frac{\beta}{2\pi}\right)^{D/2} \exp \left \{-\dfrac{\beta}{2} ||\boldsymbol{q}-\boldsymbol{y}|| \right \}$$



where $\beta^{-1}$ is the variance of the distribution. Integrating over $x$,  the marginal likelihood of the observed data for some  given value of $\boldsymbol{W}$, is obtained as: 

$$p(\boldsymbol{y}|\boldsymbol{W}, \beta) = \int p(\boldsymbol{y}|\boldsymbol{x}, \boldsymbol{W}, \beta) p(\boldsymbol{x})d\boldsymbol{x}.$$
To enable analytical evaluation of the above integral, the prior distribution of $x$ is chosen to have the form $p(\boldsymbol{x})=\frac{1}{K}\sum_{i=1}^{K}\delta(\boldsymbol{x}-\boldsymbol{x_i})$, which expresses the sum of $K$ delta functions, each of which has a center, $x_i$ on a uniform discrete grid. (An empirical test by Lawrence (2005) showed that the performance of GTM is partly dependent on the grid size). Assuming that $\boldsymbol{y}_n$ is  i.i.d, the  log likelihood can then be expressed as

$$\boldsymbol L =   \sum_{n=1}^{N} \ln\left \{ \dfrac{1}{K}\sum_{i=1}^{K}p(\boldsymbol{y}_n|\boldsymbol{x}_{i}, \boldsymbol{W}, \beta) \right \}$$

The Expectation-Maximisation (EM) algorithm is employed in the optimisation of the model parameters, $W$ and $\beta$.

A useful modification to the standard GTM which exploits the kernel trick, has been developed by Olier et al. (2010). The new method, known as  kernel GTM(KGTM) has been shown to outperform the original GTM in the visualization and clustering of more complex  data structures like strings and graphs. In this study however, we employ the original GTM due to the relative simplicity of the datasets used.

### 4.4. T-distribution Stochastic Neighbour Embedding (tSNE)

The tSNE method (van der Maaten and Hinton, 2008)   for dimensionality reduction was developed  as an extension to the stochastic neighbour embedding (SNE) method proposed by Hinton and Roweis (2002).

The SNE framework interprets the distances between data points in a high-dimensional space as conditional probabilities. Following the notation in Maaten and Hinton (2008), for two points $\boldsymbol y _i$ and $\boldsymbol y_j$ in the observed data set, we denote by $p_{j|i}$ the probability that  $\boldsymbol y_j$ will be the neighbour of $\boldsymbol y_i$, given the value of $\boldsymbol y_i$.  It can thus be naturally interpreted that if $p_{j|i}$ is large, then the two points are located close to each other in the data space. The conditional probability is Gaussian with center $\boldsymbol y_i$ and variance $\sigma^2$:
$$p_{j|i}= \dfrac{\exp(||\boldsymbol y_i-\boldsymbol y_j||^2/2\sigma_i^2)}{\sum_{k\neq i}\exp(||\boldsymbol y_i-\boldsymbol y_k||^2/2\sigma_i^2)}. $$

See van der Maaten and Hinton (2008) for details on choice of $\sigma^2$. Denoting the low-dimensional counterparts of  $y_i$ and $y_j$ by $\boldsymbol x_i$ and $\boldsymbol x_j$ respectively, the conditional probability of $ \boldsymbol x_j$ given $\boldsymbol x_i$ can be expressed as:

$$q_{j|i}= \dfrac{\exp(||\boldsymbol x_i-\boldsymbol x_j||^2/2\sigma_i^2)}{\sum_{k\neq i}\exp(||\boldsymbol x_i-\boldsymbol x_k||^2/2\sigma_i^2)}.    $$

Ideally,  we  would expect equality between  the distributions $p_{j|i}$ and $q_{j|i}.$ Therefore, SNE minimises $\sum_i KL(P_i||Q_i)=\sum_i \sum_j p_{j|i}\log \frac{p_{j|i}}{q_{j|i}}$ by gradient descent, where $KL(P_i||Q_i)$   represents  the Kullback-Leibler divergence between the two distributions. 

SNE has two main drawbacks: 1) the cost function is intractable in most cases and 2) the use of the Gaussian distribution for modelling $q_{j|i}$ leads to a phenomenon called "overcrowding", where points are clustered close together on the two-dimensional map  due to a significant reduction in area.  

The tSNE addresses these two challenges: the difficult-to-optimize cost function is replaced with one which has simpler gradients by introducing symmetry in the conditional probabilities  and $q_{j|i}$ is designed to have a Student t-distribution with heavy-tails in order to check overcrowding and enable easier optimization (van der Maaten and Hinton, 2008).

### 4.5. Multi-dimensional Scaling (MDS)

Multi-dimensional scaling is the term used to describe the broad class of dimension reduction methods which deduce the location of points on a low dimensional space using only some measure of the proximity (similarity) between those points in a high dimensional space (Kruskal and Wish, 1978). Thus, in the visualisation map produced by MDS, points which have a smaller proximity value are  located close to each other and less similar points  are  far apart.

The general concept of MDS can be explained as follows: let $\delta_{ij}$ denote the  proximity value between observed $D$-dimensional points $i$ and $j$; $i,j \in (1,...,N)$, where $N$ is the number of data points. Then, the values of  $\delta_{ij}$ form an $N\times N$ matrix, $\Lambda$. Clearly,  $\delta_{ii}=0$ for all $i$.  Given $\Lambda$, MDS aims to find a set  of vectors $(\boldsymbol x_1,...,\boldsymbol x_N) \in \mathbb R^d; d<<D$,  such that  the stress,

$$A = \sqrt{\dfrac{\sum_i \sum_j[f(\boldsymbol x_i, \boldsymbol x_j)- \delta_{ij}]^2}{\sum_i \sum_j \delta_{ij}^2}}. $$

is minimised.  The denominator in the above expression acts as a scale factor, constraining $A$  to the set $[0,1]$ ( Kruskal and Wish, 1978).  The transformation $f$ is chosen to be  monotonic, in order to preseve the ordering in the original data. 

The two types of MDS techniques - metric (classic) and non-metric -  differ in terms of the manner in which $f$ is defined, which is in turn based on the data type  being considered (i.e. metric or non-metric data). In this report, we employ the metric MDS for all comparisons since we make use of non-binary data. 

## 5. Simulations

### 5.1 Comparison metrics

As performed in the comparative review by Van der Maaten, the different techniques of dimensionality reduction are tested on simulated and real datasets and their performance is measured using the classification error of the one-nearest neighbor classifier and two measures of the preservation of the local structure, the trustworthiness and the continuity measures, defined as following:

$$ T(k) = 1 - \frac 2 {nk(2n-3k-1)} \sum_{n=1}^N \sum_{j \in U_n^{(k)}} r(n,j) - k $$ 
$$ C(k) = 1 - \frac 2 {nk(2n-3k-1)} \sum_{n=1}^N \sum_{j \in V_n^{(k)}} \hat{r}(n,j) - k $$
where $ U_n^{(k)}$ (resp.  $V_n^{(k)}$) is the set of points that are amongst the $k$ nearest neighbors of the point $n$ in the latent space (resp. data space) but not in the data space (resp. latent space) and $r(n,j)$ (resp.  $\hat{r}(n,j)$) is the rank of the point $j$ in the neighborhood of $n$ in the data space (resp. latent space).

The trustworthiness thus penalizes the points that are "too close" in the latent space, compared to the original one, whereas the continuity penalizes the points that are "too far away" in the latent space. 

We also compare the computation time of the methods.

### 5.2 Simulated dataset : the Swiss roll

Each observation $y_n \in \mathbf{R}^3$ is generated according to the following process:
$$ p_n, q_n \sim \mathcal{U}[0,1]$$
$$ t_n = \frac {3 \pi} 2 (1 + 2p_n)$$
$$ y_n = (t_n\cos t_n, t_n\sin t_n, 30 q_n) + \epsilon_n$$
where $\epsilon_n \sim \mathcal{N}(0,\sigma^2)$. <br/>

The intrisic dimensionality is then $Q = 2$ and we separate the data points into 2 classes according to the following rule:
$$ c_n = \lfloor t_n / 2 \rfloor + \lfloor 30 q_n / 12 \rfloor + 2 \mod 2 $$
We use $N = 1000$ data points.

The figures below are the 3D representation of the data points (left panel) and the 2D visualization of the 2 instrinsic dimensions (right panel).

In [12]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='swiss_roll_3d.png'></td><td><img src='swiss_roll_2d.png'></td></tr></table>"))

,


We use and compare different variants of the GP-LVM: <br/>
(a) the simple PCA (top,left) <br/>
(b) the Dual Probabilistic PCA (GP-LVM with a linear kernel) (top,middle) <br/>
(c) the GP-LVM with the RBF kernel with bias and white noise (top,right) <br/>
(d) the sparse GP-LVM with the previous kernel and p = 30 inducing variables (bottom,left) <br/>
(e) the Bayesian GP-LVM with the ARD kernel and p = 30 (bottom,right) <br/>

These models are implemented in Python in the library `GPy`.

We also choose $k = 5$ to compute the trustworthiness and the continuity measures.

The resulting 2D visualizations are displayed in the following figures and the results are summarized in the following table.



| | PCA        | DPPCA | GP-LVM           | Sparse GP-LVM | Bayesian GP-LVM |
| ------------- |:-------------:| -----:| 
| classification error (%)| 27.7 | 26.7 | 8.0 |8.4| 7.0 |
| trustworthiness    | 0.8851   | 0.8873 | 0.9967 | 0.9975| 0.9887 |
| continuity | 0.9988| 0.9960 | 0.9956|0.9792| 0.9700 |
| wall time | 115 ms | 42 s | 8 min 38 s | 2 min 12 s | 9 min 53 s|

In [13]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='swiss_roll_pca.png'></td><td><img src='swiss_roll_lin.png'></td><td><img src='swiss_roll_rbf.png'></td></tr></table>"))
display(HTML("<table><tr><td><img src='swiss_roll_sparse.png'></td><td><img src='swiss_roll_bayesian.png'></td></tr></table>"))

,,


,


As expected, we observe that the linear models do not perform well in this case. Although the non-linear GP-LVM perform quite well and quite efficiently with the sparse approximation, they do not reach the performances of the best methods compared in Maaten's review such as Isomap, Diffusion Maps or Local Linear Embeddings, which achieve a classification error of less than 4% on a test set.

The following techniques of dimensionality reduction were also tested on this same dataset:
1. t-SNE 
2. MDS
3. GTM
4. Kernel PCA with polynomial, sigmoid and RBF kernel: <br/>
(a) KPCA with polynomial kernel (degree 2) <br/>
(b) KPCA with sigmoid kernel (default values) <br/>
(c) KPCA with RBF kernel (gamma = 0.5) <br/>
5. Autoencoder with 3 hidden layers

The two first methods are implemented in the `scikit-learn` library and the third one in `ugtm`, the kernel PCA in `KernelPCA` and the autoencoder in `keras`. 

The results are in the following table.

| | Sparse GP-LVM | tSNE |MDS| GTM |KPCA poly | KPCA sigmoid | KPCA RBF | Autoencoders|
| ------------- |:-------------:| -----:| 
| classification error (%)|8.4 |  28.1| 30.2 | 33.8| 25.4 | 13.9 | 25.3 |23.9
| trustworthiness    | 0.9975| 0.9996 | 0.8949| 0.9634| 0.8770 | 0.9473 | 0.8652 | 0.8704 |
| continuity |0.9700| 0.9916 |  0.9931| 0.9894| 0.9911 | 0.9962| 0.9924 | 0.9930 |
| wall time | 2 min 12 s |6.14 s  |3 s  | 1.4 s| 35.1 ms | 60.7 ms | 75.1 ms| 638 ms

In [14]:
display(HTML("<table><tr><td><img src='swiss_roll_emmanuelle.png'></td></tr></table>"))

""


In [15]:
display(HTML("<table><tr><td><img src='swiss_roll_kpca.png'></td></tr></table>"))

""


In [16]:
display(HTML("<table><tr><td><img src='swiss_roll_ae.png'></td></tr></table>"))

""


While the sigmoid kernel and the RBF kernel with gamma=0.01 led to a lower error, the RBF kernel with gamma equals 0.5 seems to produce a more clear visualisation. The autoencoder with 2 hidden layers does not give excellent results so increasing the number of layers could perhaps improve its performance.


The GP-LVM outperforms all the other techniques for this dataset, although it seems to have a higher computation cost. 

### 5.3 Real dataset : Oil flow

In this real dataset, the observations $y_n \in \mathbf{R}^{12}$ have labels corresponding to three types of flows: homogeneous, annular or stratified. We use $N = 1000$ data points and try to visualize them in two dimensions. <br/>

All the methods described previously are compared and the numerical results are summarized in the following table.

| | PCA | DPPCA |       GP-LVM           | Sparse GP-LVM | Bayesian GP-LVM | KPCA poly | KPCA sigmoid | KPCA RBF | Autoencoders |tSNE |MDS | GTM
| ------------- |:-------------:| -----:| 
| classification error (%)| 27 | 26.5 | 0.0 | 0.0 | 0.1 | 32.2 | 35.2 | 15.2 | 14.4 | 0.3 | 28.9| 4.3
| trustworthiness    | 0.9207 | 0.9195 | 0.9849 | 0.9969| 0.9971| 0.9275 | 0.8957 | 0.8182 | 0.9143| 0.9985 | 0.9306| 0.9898
| continuity | 0.9921 | 0.9912 |0.9993 | 0.9769 | 0.9975 | 0.9886 | 0.9910 | 0.9578 | 0.9707 | 0.9964|0.9248|0.9836
| computation time | 350 ms | 8.06 s | 8 min 38 s | 2 min 24 s | 3 min 34 s | 32.6 ms |  78.5 ms | 77.4 ms | 276 ms|6.41 s|3.02 s| 1.48 s

In [17]:
print("GP-LVM methods")
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='oil_pca.png'></td><td><img src='oil_lin.png'></td><td><img src='oil_rbf.png'></td></tr></table>"))
display(HTML("<table><tr><td><img src='oil_sparse.png'></td><td><img src='oil_bayesian.png'></td></tr></table>"))

GP-LVM methods


,,


,


The non-linear GP-LVMs perform well in this situation, although the Bayesian GP-LVM seems to be a bit better at separating the different classes.These results are consistent with those of Titsias and Lawrence.

In [18]:
display(HTML("<table><tr><td><img src='oil_emmanuelle.png'></td></tr></table>"))

""


 From the visualisations, it is clear that the tSNE method is more efficient in recovering the intrinsic  pattern of the underlying data set. However, it appears to perform better in simulated data than in real data.  Also, both MDS and GTM methods perform averagely better  on the natural data than they do on  artificial data. A suggested area for research might be to investigate why this is the case. It is reasonable to attribute the better performance of tSNE to the  non-convexity of its objective function  which makes the method free from the challenges of convex optimization such as numerical problems in optimisation (van der Maaten et al; 2009).

In [19]:
print("Kernel PCA with different kernels and hyperparameters")
display(HTML("<table><tr><td><img src='oil_kpca.png'></td></tr></table>"))

Kernel PCA with different kernels and hyperparameters


""


In [20]:
print("Autoencoder")
display(HTML("<table><tr><td><img src='oil_ae.png'></td></tr></table>"))

Autoencoder


""


 The KPCA method worked best by choosing an RBF kernel with gamma 0.5 which produced a visually clear partition of the data and a classification error of 0.152. However, the result was very dependent on the value of gamma and perhaps a more appropriate value could have been chosen by using cross-validation techniques. Different runs of the autoencoder produced very different plots with varying metric values. In general, the error was similar to the RBF kernel but much higher on some occasions. It would have been of interest to study how the number of hidden layers and the type of activation function affected the performance of the autoencoder. We used a linear activation function for the middle layer and a sigmoid function for the rest but perhaps the use of different activation functions could have increased the performance of this method.

Once again, the GP-LVM gives the best results in our comparison study, but t-SNE also performs quite similarly. 

## 6. Discussion

Representing high dimensional data on a 2-dimensional visualization can be very challenging when no prior information on the data is known. Amongst the different non-linear models using non-convex objective function optimization, the Gaussian Process Latent Variable Model is a quite robust and flexible framework, than can also be scalable using some sparse approximation. Moreover, it can be adapted to dynamical models for motion tracking for instance. 

## References

 Neil Lawrence. Gaussian process latent variable models for visualisation of high dimensional data. *Advances in neural information processing systems*, pp. 329-336, 2004.
  
   Neil Lawrence. Probabilistic non-linear principal component analysis with Gaussian process latent variable models. *Journal of machine learning research* 6 (Nov):1783-1816, 2005.
  
   Michalis K. Titsias, Neil Lawrence. Bayesian Gaussian process latent variable models. *Proceedings of the Thirteenth International Conference on Artificial Intelligence and Statistics, PMLR*, 9:844-851, 2010.
  
  Christopher M. Bishop, Marcus Svensén and Christopher Williams. GTM: the Generative Topographic Mapping.  *Neural Computation*, 10(1):215-234, 1998.
  
   Schulz,  Eric, Maarten Speekenbrink, Andreas Krause. A tutorial on Gaussian process regression: Modelling, exploring, and exploiting functions. *Journal of Mathematical Psychology*, 85:1-16, 2018.
   
   Gaspar, H.A., 2018. ugtm: A Python Package for Data Modeling and Visualization Using Generative Topographic Mapping. *Journal of Open Research Software*, 6(1):, 2018. DOI: http://doi.org/10.5334/jors.235
   
   Laurens Van Der Maaten, Eric Postma, Jaap Van den Herik. Dimensionality Reduction: a Comparative Review. *J Mach Learn Res*, 10:66-71, 2009.
  
  Yarin Gal, Mark Van Der Wilk and Carl Edward Rasmussen. Distributed variational inference in sparse Gaussian process regression and latent variable models. *Advances in neural information processing systems*, pp. 3257-3265, 2014.
  
  De G Matthews, G Alexander, Mark Van Der Wilk, Tom Nickson, Keisuke Fujii, Alexis Boukouvalas, Pablo León-Villagrá, Zoubin Ghahramani and James Hensman. GPflow: A Gaussian process library using TensorFlow. *The Journal of Machine Learning Research*, 18 (1):1299-1304, 2017.
  
  Sumon Ahmed, Magnus Rattray, Alexis Boukouvalas. GrandPrix: Scaling up the Bayesian GPLVM for single-cell data. *Bioinformatics*, 35(1):47-54, 2018.
  
 Kaspar Märtens, Kieran R Campbell, Christopher Yau. Covariate Gaussian Process Latent Variable Models. arXiv preprint arXiv:1810.06983, 2018.
 
 Olier  Iván, Alfredo Vellido and Jesús Giraldo. Kernel generative topographic mapping. ESANN, 2010. 
 
  Laurens van der Maaten, and Geoffrey Hinton. Visualizing data using t-SNE. *Journal of machine learning research*, pp 2579-2605, 2008.
  
  Joseph B. Kruskal  and Wish, Myron.  Multidimensional Scaling. Sage University Papers Series. *Quantitative Applications in the Social Sciences* ; pp. 07-011, 1978.
  
 Geoffrey E.  Hinton and Sam T. Roweis. Stochastic neighbor embedding. *In Advances in neural information processing systems*, pp. 857-864, 2003.